In [5]:
!pip install medspacy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.4/111.4 kB 3.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.4/67.4 kB 5.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 71.1/71.1 kB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 8.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 29.2 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 41.3 MB/s eta 0:00:00:00:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 575.5/575.5 kB 33.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.5/320.5 kB 23.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.7/311.7 kB 22.1 MB/s eta 0:00:00
  Created wheel for medspacy: filename=medspacy-1.1.2-py3-none-any.whl size=147920 sha256=f

In [6]:
import numpy as np 
import pandas as pd
import os

import matplotlib.pyplot as plt
plt.rcParams.update({'font.size': 10})

In [14]:
df = pd.read_csv('/kaggle/input/morticd10/codes.csv')

#structuring
df.columns = ['prefix_codes', 'decimal', 'codes', 'description', 'long_description', 'label']

In [15]:
category_distribution = df['prefix_codes'].value_counts()
lower_threshold = 5
upper_threshold = 10
filtered_categories = category_distribution[(category_distribution > lower_threshold) & (category_distribution < upper_threshold)]

# print(filtered_categories)

In [16]:
new_df = df[df['prefix_codes'].isin(filtered_categories.index)]

In [17]:
new_df = new_df.sample(frac=1, random_state=23)
print(f"Size: {len(new_df)}")
new_df.head()

Size: 15036


,prefix_codes,decimal,codes,description,long_description,label
28706,S32463,D,S32463D,"Displ assoc transv/post fx unsp acetab, 7thD",Displaced associated transverse-posterior frac...,Displaced associated transverse-posterior frac...
18763,N88,0,N880,Leukoplakia of cervix uteri,Leukoplakia of cervix uteri,Other noninflammatory disorders of cervix uteri
4627,G11,8,G118,Other hereditary ataxias,Other hereditary ataxias,Hereditary ataxia
3707,E7524,1,E75241,Niemann-Pick disease type B,Niemann-Pick disease type B,Niemann-Pick disease
31511,S42456,A,S42456A,"Nondisp fx of lateral condyle of unsp humerus,...",Nondisplaced fracture of lateral condyle of un...,Nondisplaced fracture of lateral condyle of un...


In [18]:
def normalise_text (text):
    text = text.lower() # lowercase
    text = text.replace(r"\#","") # replaces hashtags
    text = text.replace(r"@","")
    text = text.replace(r"[^A-Za-z0-9()!?\'\`\"]", " ")
    text = text.replace("\s{2,}", " ")
    return text

df['long_description'] = df['long_description'].apply(normalise_text)
df.head()

,prefix_codes,decimal,codes,description,long_description,label
0,A00,1,A001,"Cholera due to Vibrio cholerae 01, biovar eltor","cholera due to vibrio cholerae 01, biovar eltor",Cholera
1,A00,9,A009,"Cholera, unspecified","cholera, unspecified",Cholera
2,A010,0,A0100,"Typhoid fever, unspecified","typhoid fever, unspecified",Typhoid fever
3,A010,1,A0101,Typhoid meningitis,typhoid meningitis,Typhoid fever
4,A010,2,A0102,Typhoid fever with heart involvement,typhoid fever with heart involvement,Typhoid fever


In [13]:
import medspacy
from medspacy.ner import TargetRule
from medspacy.visualization import visualize_ent

/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: libtensorflow_io.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io.so: undefined symbol: _ZTVN10tenso

In [ ]:
#

In [21]:
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

X = df[['long_description']]
y = df['prefix_codes']


encoder = LabelEncoder()
y = encoder.fit_transform(y)

# Split the dataset into a train and temporary set
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.2, random_state=23)

# Split the temporary set into validation and train set
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=23)

In [33]:
import torch
from torch.utils.data import Dataset, DataLoader
import torchtext

from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained('emilyalsentzer/Bio_ClinicalBERT')
model = AutoModelForSequenceClassification.from_pretrained('emilyalsentzer/Bio_ClinicalBERT', 
                                                           num_labels=len(new_df['prefix_codes'].unique()))


Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [43]:
print(f"Input Ids: {tokenizer(X_train.iloc[100].long_description, truncation=True, padding=True)['input_ids']}")
print(f"Attention Masks: {tokenizer(X_train.iloc[100].long_description, truncation=True, padding=True)['attention_mask']}")

Input Ids: [101, 12844, 2629, 1104, 6302, 21359, 4487, 1732, 10885, 3269, 117, 5576, 24951, 15842, 117, 4194, 8107, 102]
Attention Masks: [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]


In [24]:
class CustomDataset(Dataset):
    def __init__(self, X, y, tokenizer, max_length):
        self.data = []
        for i in zip(X, y):
            text, target = i[0], i[1]
            self.data.append((text, int(target)))
            self.tokenizer = tokenizer
            self.max_length = max_length
            

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        text, target = self.data[idx]
        inputs = self.tokenizer(text, padding='max_length', truncation=True, 
                                max_length=self.max_length, return_tensors='pt')
        inputs_ids = inputs['input_ids'].squeeze(0)
        attention_masks = inputs['attention_mask'].squeeze(0)
        return inputs_ids, attention_masks, target 

In [46]:
sentences = new_df["long_description"]
sentences_tokenized = [tokenizer.tokenize(sentence) for sentence in sentences ]

print('Max sentence length: ', max([len(sen) for sen in sentences_tokenized]))
max_len = max([len(sen) for sen in sentences_tokenized])

Max sentence length:  41


In [48]:
train_dataset = CustomDataset(X_train, y_train, tokenizer, max_len)
val_dataset = CustomDataset(X_test, y_test, tokenizer, max_len)

batch_size = 2
train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True)

In [49]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

loss_function = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)

In [50]:
model = model.to(device)

In [52]:
for epoch in range(3):  # num_train_epochs = 3
    total_loss = 0
    model.train()  # put the model in training mode
    
    for batch in train_dataloader:
        # move batch to device
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch
        
        # forward pass
        outputs = model(b_input_ids, attention_mask=b_input_mask, labels=b_labels)
        loss = outputs[0]
        
        # backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        total_loss += loss.item()
    
    avg_train_loss = total_loss / len(train_loader)
    print("Average training loss: {0:.2f}".format(avg_train_loss))

IndexError: Target 17693 is out of bounds.